# Exploratory Data analysis : Shopping trands data set

## Libraries

In [30]:
import pandas as pd

## Overview of the data

In [31]:
df = pd.read_csv("shopping_trends_updated.csv")
print(df.shape)
print(df.columns)

(3900, 18)
Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')


In [32]:
# Statistic description of quantitative variables
print(df.loc[:,["Age", "Review Rating", "Previous Purchases", "Purchase Amount (USD)"]].describe())

               Age  Review Rating  Previous Purchases  Purchase Amount (USD)
count  3900.000000    3900.000000         3900.000000            3900.000000
mean     44.068462       3.749949           25.351538              59.764359
std      15.207589       0.716223           14.447125              23.685392
min      18.000000       2.500000            1.000000              20.000000
25%      31.000000       3.100000           13.000000              39.000000
50%      44.000000       3.700000           25.000000              60.000000
75%      57.000000       4.400000           38.000000              81.000000
max      70.000000       5.000000           50.000000             100.000000



Checking if there are any null values

Checking if there are any duplicated values

Describe each variable with matplotlib

## Questions:
### 1. What is the average age of customers ?
### 2. What is the most common item purchased ?
### 3. What is the total purchase amount for each category ?
### 4. How many customers have opted for the Subscription ?
### 5. What is the total purchase amount for customers in each location ? Make a world cloud
### 6. KDE plot for Purchase Amount by Season
### add more questions